In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [4]:
## load the ANN trained model , scaler pickle, onehot encoder
model = load_model('model.h5')

## load(read) the emcoder and scalar
with open('label_encoder_gender.pk1','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geography.pk1', 'rb') as file:
    onehot_encoder_geography = pickle.load(file)

with open('scalar.pk1', 'rb') as file:
    scalar = pickle.load(file)
    

In [5]:
# Example input
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [6]:
## convert into Dataframe
import pandas as pd

# Single-row DataFrame
input_df = pd.DataFrame([input_data])


In [7]:
## Encode categorival variables
# Encode Gender using the label encoder fitted on training data
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

# One-hot encode Geography using the encoder fitted on training data
geography_encoded = onehot_encoder_geography.transform(input_df[['Geography']]).toarray()
geography_df = pd.DataFrame(
    geography_encoded,
    columns=onehot_encoder_geography.get_feature_names_out(['Geography'])
)

# Drop original Geography column and concatenate encoded
input_df = pd.concat([input_df.drop('Geography', axis=1), geography_df], axis=1)


In [8]:
## Reorder columns to match training data
# Ensure columns match scaler input
expected_columns = scalar.feature_names_in_
input_df = input_df[expected_columns]


In [9]:
## Scale the input
input_scaled = scalar.transform(input_df)
input_scaled


array([[-0.47912542,  0.91678285,  0.09830529, -0.71248217, -0.26873812,
         0.78043148,  0.64635082,  0.96914252, -0.84682956,  0.99468081,
        -0.58607517, -0.56451866]])

In [10]:
prediction= model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step


array([[0.04387524]], dtype=float32)

In [11]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.043875244)

In [ ]:
if prediction_prob > 0.5:
    print(f"Customer is likely to churn!")
else:
    print(f"Customer is not likely to churn!")

Customer is not likely to churn!.
